In [28]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:

import tensorflow as tf 
tf.test.gpu_device_name() 


'/device:GPU:0'

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')



In [0]:
df=pd.read_csv('/content/drive/My Drive/stackoverflow/Train.csv')

In [0]:
df.shape

(6034195, 4)

In [0]:
df.head()

,Id,Title,Body,Tags
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents


To find duplicates in the dataset groups rows based on title,body and tags

In [0]:
df_d = df.groupby(['Title', 'Body','Tags']).size().reset_index(name='dup_count')

In [0]:
df_d.head()

,Title,Body,Tags,dup_count
0,Implementing Boundary Value Analysis of S...,<pre><code>#include&lt;iostream&gt;\n#include&...,c++ c,1
1,Dynamic Datagrid Binding in Silverlight?,<p>I should do binding for datagrid dynamicall...,c# silverlight data-binding,1
2,Dynamic Datagrid Binding in Silverlight?,<p>I should do binding for datagrid dynamicall...,c# silverlight data-binding columns,1
3,java.lang.NoClassDefFoundError: javax/serv...,"<p>I followed the guide in <a href=""http://sta...",jsp jstl,1
4,java.sql.SQLException:[Microsoft][ODBC Dri...,<p>I use the following code</p>\n\n<pre><code>...,java jdbc,2


In [0]:
print("total duplicate rows  ", (df.shape[0]-df_d.shape[0]))

total duplicate rows   1827888


Duplicate rows count

In [0]:
df_d["tag_count"] = df_d["Tags"].apply(lambda text: len(text.split(" ")))

In [0]:
df_d.head()

,Title,Body,Tags,dup_count,tag_count
0,Implementing Boundary Value Analysis of S...,<pre><code>#include&lt;iostream&gt;\n#include&...,c++ c,1,2
1,Dynamic Datagrid Binding in Silverlight?,<p>I should do binding for datagrid dynamicall...,c# silverlight data-binding,1,3
2,Dynamic Datagrid Binding in Silverlight?,<p>I should do binding for datagrid dynamicall...,c# silverlight data-binding columns,1,4
3,java.lang.NoClassDefFoundError: javax/serv...,"<p>I followed the guide in <a href=""http://sta...",jsp jstl,1,2
4,java.sql.SQLException:[Microsoft][ODBC Dri...,<p>I use the following code</p>\n\n<pre><code>...,java jdbc,2,2


In [0]:
df_d['tag_count'].value_counts()

3    1206157
2    1111706
4     814996
1     568290
5     505158
Name: tag_count, dtype: int64

**Observation:**
1.   Range of tags for question will be from 1 to 5.
2.   Most of the questions have 3 tags.
3.   Very few questions have 5 tags


In [0]:
df_d.to_csv('/content/drive/My Drive/stackoverflow/temp1.csv')

In [0]:
df_d=pd.read_csv('/content/drive/My Drive/stackoverflow/temp1.csv')

In [0]:
df_d.shape

(4206307, 6)

In [0]:
df_d.head()

,Unnamed: 0,Title,Body,Tags,dup_count,tag_count
0,0,Implementing Boundary Value Analysis of S...,<pre><code>#include&lt;iostream&gt;\n#include&...,c++ c,1,2
1,1,Dynamic Datagrid Binding in Silverlight?,<p>I should do binding for datagrid dynamicall...,c# silverlight data-binding,1,3
2,2,Dynamic Datagrid Binding in Silverlight?,<p>I should do binding for datagrid dynamicall...,c# silverlight data-binding columns,1,4
3,3,java.lang.NoClassDefFoundError: javax/serv...,"<p>I followed the guide in <a href=""http://sta...",jsp jstl,1,2
4,4,java.sql.SQLException:[Microsoft][ODBC Dri...,<p>I use the following code</p>\n\n<pre><code>...,java jdbc,2,2


In [0]:
df_d.shape

(4206307, 6)

 **Analysis of tags**

In [0]:
ll=[x.split() for x in df_d['Tags'].unique()]

In [0]:
flatten = [item for l in ll for item in l]

In [32]:
abs=set(flatten)
print("number of unique tags:", len(abs))

number of unique tags: 42048


In [0]:
tags = list(abs)

In [0]:
tag_df = pd.DataFrame({'col':flatten})

In [0]:
tag_df = tag_df.groupby(['col']).size().reset_index(name='count')

In [36]:
tag_df.shape

(42048, 2)

In [0]:
tag_df.head()

,col,count
0,.a,18
1,.app,32
2,.asp.net-mvc,1
3,.aspxauth,21
4,.bash-profile,119


In [0]:
tag_df_sorted = tag_df.sort_values(['count'], ascending=False)
tag_counts = tag_df_sorted['count'].values

In [38]:
tag_df_sorted[:10]

,col,count
4337,c#,163396
18069,java,154137
27249,php,118622
18157,javascript,108356
1234,android,96272
18608,jquery,81494
4346,c++,80126
22,.net,67171
29101,python,67033
17643,iphone,62965


**Observation**


1.   Top ten tags were related to programming languages,so most of the questions in the stackoverflow website were related to programming languages. 



In [0]:
df_d_lim=df_d.sample(n=500000)

In [0]:
df_d_lim_pre=df_d_lim[['Title','Body','Tags']]

In [41]:
df_d_lim_pre.head()

,Title,Body,Tags
2593497,Quick Question About NSFetchRequest and Relati...,"<p>In my Core Data model, I have an entity Ses...",iphone objective-c xcode nsfetchedresultscontr...
4147004,update Window media player Param,<p>I added a WMP to my page as follow:</p>\n\n...,jquery wmp
769195,Django: Filter for many to many field gives to...,<p>have the following four models</p>\n\n<p><s...,python django query many-to-many
998832,Force context menu to appear for form inputs,<p>I'm trying to develop a ff addon that allow...,firefox firefox-addon firefox-addon-sdk
4117736,structure of modules over $\mathbb{Z}^n$,"<p>As we know, there's a structure theorem for...",ring-theory modules


**Data cleaning**


1.   Removing html tags.
2.   Removing stop words.
3.   Convert all the characters into small letters.
4.   Use SnowballStemmer to stem the words.
5.   Remove code snippets from the body.
6.   Concatenate title and body into single column.



In [42]:
def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(data))
    return cleantext
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")
df_d_lim_pre['final']=''
temp_l=[]
for i in range(len(df_d_lim_pre)): 
  temp=df_d_lim_pre.iloc[i,1]
  title=df_d_lim_pre.iloc[i,0]
  temp=re.sub('<code>(.*?)</code>', '',temp, flags=re.MULTILINE|re.DOTALL)
  temp=striphtml(temp.encode('utf-8'))
  title=title.encode('utf-8')
  temp=str(title)+" "+str(temp)
  temp=re.sub(r'[^A-Za-z]+',' ',temp)
  words=word_tokenize(str(temp.lower()))
  temp=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
  temp_l.append(temp)
df_d_lim_pre['final']=temp_l


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
new_df=df_d_lim_pre[['final','Tags']]

In [0]:
new_df.shape

(500000, 2)

In [0]:
new_df.head()

,final,Tags
4044210,python write file keep get write close file er...,python
477329,casperj phantomj access anchor href javascript...,javascript events href phantomjs casperjs
1607821,observ custom object chang even extend let say...,java inheritance fields onchange listeners
2778595,stathread main method want appli stathread att...,c#
2431386,page stretch amaz titl basic explain page clie...,html frontend web-frontend


In [0]:
new_df.to_csv('/content/drive/My Drive/stackoverflow/temp2.csv')

In [0]:
new_df=pd.read_csv('/content/drive/My Drive/stackoverflow/temp2.csv')

**Considering computational limitation sampled only 50000 questions**

In [0]:
new_df2=new_df.sample(n=50000)

***Spliting the data into train and test(80:20)***

In [0]:
xTrain, xTest= train_test_split(new_df2.values,test_size = 0.2, random_state = 0)

In [46]:
xTrain.shape

(40000, 2)

In [0]:
YTRAIN=xTrain[:,2]
YTEST=xTest[:,2]
XTRAIN=xTrain[:,1]
XTEST=xTest[:,1]

**Using tfidf converted that text data present in train and test to numeric data**

In [0]:
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=60000, smooth_idf=True,norm='l2',ngram_range=(1,3))
x_train_multilabel = vectorizer.fit_transform(XTRAIN)
x_test_multilabel = vectorizer.transform(XTEST)

**Analysis on the tags present in train data which is 40000 questions.**

In [0]:
vectorizer1 = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
YTRAIN_NEW = vectorizer1.fit_transform(YTRAIN)

In [0]:
abc=vectorizer1.get_feature_names()

In [50]:
abc[:10]

['.app',
 '.bash-profile',
 '.exe',
 '.htaccess',
 '.htpasswd',
 '.lib',
 '.mov',
 '.net',
 '.net-1.1',
 '.net-2.0']

In [13]:
print("Total number of unique tags present in train data :",len(abc))

Total number of unique tags present in train data : 12648


**If we consider all the tags ppresent in train data then we have to train nearly 12580 models on complete which is not computationally efficient, so we are compromising on the number tags based on the partial coverage of the questions.**

In [0]:
def top_tags(n):
    t=YTRAIN_NEW.sum(axis=0).tolist()[0]
    tags=sorted(range(len(t)), key=lambda i: t[i], reverse=True)
    tags_sel=YTRAIN_NEW[:,tags[:n]]
    return (tags_sel,[abc[i] for i in tags[:n]])

def partial_coverage(n):
    tags_matrix,_ = top_tags(n)
    x=tags_matrix.sum(axis=1)
    return (np.count_nonzero(x==0))

In [0]:
percentage_of_questions= []
total_tags=YTRAIN_NEW.shape[1]
total_qs=XTRAIN.shape[0]
for i in range(500, total_tags,100):
    percentage_of_questions.append(np.round(((total_qs-partial_coverage(i))/total_qs)*100,3))

In [53]:
for i,j in enumerate(percentage_of_questions):
  print("with ",500+i*100,"tags we are covering ",j,"% of questions")

with  500 tags we are covering  90.325 % of questions
with  600 tags we are covering  91.59 % of questions
with  700 tags we are covering  92.558 % of questions
with  800 tags we are covering  93.282 % of questions
with  900 tags we are covering  93.848 % of questions
with  1000 tags we are covering  94.375 % of questions
with  1100 tags we are covering  94.812 % of questions
with  1200 tags we are covering  95.19 % of questions
with  1300 tags we are covering  95.5 % of questions
with  1400 tags we are covering  95.78 % of questions
with  1500 tags we are covering  96.083 % of questions
with  1600 tags we are covering  96.28 % of questions
with  1700 tags we are covering  96.48 % of questions
with  1800 tags we are covering  96.68 % of questions
with  1900 tags we are covering  96.815 % of questions
with  2000 tags we are covering  96.962 % of questions
with  2100 tags we are covering  97.098 % of questions
with  2200 tags we are covering  97.288 % of questions
with  2300 tags we are 

**Observation**


1.   By choosing top 500 tags we can cover 90% of the questions in train data.


In [0]:
YTRAIN_NEW2,features= top_tags(500)

In [18]:
YTRAIN_NEW2

<40000x500 sparse matrix of type '<class 'numpy.int64'>'
	with 72489 stored elements in Compressed Sparse Row format>

**using bag of words and vocabulary as the top 500 tags choosen, convereted tags columns into sparse matrix which is suitable for modeling.**

In [55]:
vectorizer2 = CountVectorizer(vocabulary=features,binary='true')
multilabel_y2 = vectorizer2.fit_transform(YTRAIN)
print("train label shape", multilabel_y2.shape)
newytest= vectorizer2.transform(YTEST)
print("test label shape", newytest.shape)

train label shape (40000, 500)
test label shape (10000, 500)


**Model imolementation(logistic regression using one vs rest)**

In [61]:
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.0001, penalty='l1'), n_jobs=-1)
classifier.fit(x_train_multilabel, multilabel_y2)
predictions = classifier.predict(x_test_multilabel)
f1 = f1_score(newytest, predictions,average='micro')
print(f1)

0.3280925541382379


**HYPER PARAMETER TUNING**

In [62]:
params = {'estimator__loss':['hinge','log'],'estimator__penalty':['l1','l2'],'estimator__alpha':[0.0000001,0.000001,0.00001,0.0001, 0.001, 0.01, 0.1]}
clf = OneVsRestClassifier(SGDClassifier(), n_jobs=-1)
model = GridSearchCV(clf, param_grid=params, scoring = 'f1_micro', cv=2, verbose=54)
model.fit(x_train_multilabel, multilabel_y2)

Fitting 2 folds for each of 28 candidates, totalling 56 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] estimator__alpha=1e-07, estimator__loss=hinge, estimator__penalty=l1 
[CV]  estimator__alpha=1e-07, estimator__loss=hinge, estimator__penalty=l1, score=0.484, total=  48.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   48.1s remaining:    0.0s
[CV] estimator__alpha=1e-07, estimator__loss=hinge, estimator__penalty=l1 
[CV]  estimator__alpha=1e-07, estimator__loss=hinge, estimator__penalty=l1, score=0.481, total=  47.9s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.6min remaining:    0.0s
[CV] estimator__alpha=1e-07, estimator__loss=hinge, estimator__penalty=l2 
[CV]  estimator__alpha=1e-07, estimator__loss=hinge, estimator__penalty=l2, score=0.471, total=  25.8s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.0min remaining:    0.0s
[CV] estimator__alpha=1e-07, estimator__loss=hinge, estimator__penalty=l2 
[CV]  est

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  estimator__alpha=0.001, estimator__loss=log, estimator__penalty=l2, score=0.000, total=  38.6s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed: 25.0min remaining:    0.0s
[CV] estimator__alpha=0.001, estimator__loss=log, estimator__penalty=l2 
[CV]  estimator__alpha=0.001, estimator__loss=log, estimator__penalty=l2, score=0.000, total=  38.7s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 25.6min remaining:    0.0s
[CV] estimator__alpha=0.01, estimator__loss=hinge, estimator__penalty=l1 
[CV]  estimator__alpha=0.01, estimator__loss=hinge, estimator__penalty=l1, score=0.000, total=  33.3s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed: 26.2min remaining:    0.0s
[CV] estimator__alpha=0.01, estimator__loss=hinge, estimator__penalty=l1 
[CV]  estimator__alpha=0.01, estimator__loss=hinge, estimator__penalty=l1, score=0.000, total=  33.5s
[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed: 26.8min remaining:    0.0s
[CV] estimator__alpha=0.01, estimator__loss=hinge, es

GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001,
                                                                   average=False,
                                                                   class_weight=None,
                                                                   early_stopping=False,
                                                                   epsilon=0.1,
                                                                   eta0=0.0,
                                                                   fit_intercept=True,
                                                                   l1_ratio=0.15,
                                                                   learning_rate='optimal',
                                                                   loss='hinge',
                                                                   max_iter=1000,
                             

In [63]:
model.best_params_

{'estimator__alpha': 1e-06,
 'estimator__loss': 'hinge',
 'estimator__penalty': 'l1'}

In [0]:
predictions = model.predict(x_test_multilabel)

In [65]:
precision = precision_score(newytest, predictions, average='micro')
recall = recall_score(newytest,predictions,average='micro')
f1 = f1_score(newytest, predictions,average='micro')
print("PRECISION :", precision)
print("RECALL :", recall)
print("F1 SCORE:" ,f1)

PRECISION : 0.6749060707929603
RECALL : 0.4224009900990099
F1 SCORE: 0.5196011265890234


**FEATURE ENGINEERING: Considering title has more important than body, so giving more weightage to title may increase the f1 score of the model**

In [0]:
edf=df_d_lim_pre.copy()

In [0]:
def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(data))
    return cleantext
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")
edf['final']=''
temp_l=[]
for i in range(len(edf)): 
  temp=edf.iloc[i,1]
  title=edf.iloc[i,0]
  temp=re.sub('<code>(.*?)</code>', '',temp, flags=re.MULTILINE|re.DOTALL)
  temp=striphtml(temp.encode('utf-8'))
  title=title.encode('utf-8')
  temp=str(title)+" "+str(title)+" "+str(title)+" "+str(temp)
  temp=re.sub(r'[^A-Za-z]+',' ',temp)
  words=word_tokenize(str(temp.lower()))
  temp=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
  temp_l.append(temp)
edf['final']=temp_l

In [68]:
edf.head()

,Title,Body,Tags,final
2593497,Quick Question About NSFetchRequest and Relati...,"<p>In my Core Data model, I have an entity Ses...",iphone objective-c xcode nsfetchedresultscontr...,quick question nsfetchrequest relationship qui...
4147004,update Window media player Param,<p>I added a WMP to my page as follow:</p>\n\n...,jquery wmp,updat window media player param updat window m...
769195,Django: Filter for many to many field gives to...,<p>have the following four models</p>\n\n<p><s...,python django query many-to-many,django filter mani mani field give mani answer...
998832,Force context menu to appear for form inputs,<p>I'm trying to develop a ff addon that allow...,firefox firefox-addon firefox-addon-sdk,forc context menu appear form input forc conte...
4117736,structure of modules over $\mathbb{Z}^n$,"<p>As we know, there's a structure theorem for...",ring-theory modules,structur modul mathbb structur modul mathbb st...


In [0]:
new_df11=edf[['final','Tags']]
new_df211=new_df11.sample(n=50000)
xTrain11, xTest11= train_test_split(new_df211.values,test_size = 0.2, random_state = 0)
YTRAIN11=xTrain11[:,1]
YTEST11=xTest11[:,1]
XTRAIN11=xTrain11[:,0]
XTEST11=xTest11[:,0]

vectorizer11 = TfidfVectorizer(min_df=0.00009, max_features=50000, smooth_idf=True,norm='l2',ngram_range=(1,3))
x_train_multilabel11 = vectorizer11.fit_transform(XTRAIN11)
x_test_multilabel11 = vectorizer11.transform(XTEST11)
vectorizer111 = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
YTRAIN_NEW11 = vectorizer111.fit_transform(YTRAIN11)

abc1=vectorizer111.get_feature_names()



def top_tags1(n):
    t1=YTRAIN_NEW11.sum(axis=0).tolist()[0]
    tags11=sorted(range(len(t1)), key=lambda i: t1[i], reverse=True)
    tags_sel11=YTRAIN_NEW11[:,tags11[:n]]
    return (tags_sel11,[abc1[i] for i in tags11[:n]])

def partial_coverage1(n):
    tags_matrix1,_ = top_tags1(n)
    x=tags_matrix1.sum(axis=1)
    return (np.count_nonzero(x==0))


percentage_of_questions1= []
total_tags1=YTRAIN_NEW11.shape[1]
total_qs1=XTRAIN11.shape[0]
for i in range(500, total_tags1,100):
    percentage_of_questions1.append(np.round(((total_qs1-partial_coverage1(i))/total_qs1)*100,3))


In [86]:
YTRAIN_NEW211,features11= top_tags1(500)


vectorizer211 = CountVectorizer(vocabulary=features11,binary='true')
multilabel_y211 = vectorizer211.fit_transform(YTRAIN11)
print("train label shape", multilabel_y211.shape)
newytest11= vectorizer211.transform(YTEST11)
print("test label shape", newytest11.shape)



train label shape (40000, 500)
test label shape (10000, 500)


In [87]:
classifier1 = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.000001, penalty='l1'), n_jobs=-1)
classifier1.fit(x_train_multilabel11, multilabel_y211)
predictions11 = classifier1.predict(x_test_multilabel11)
precision1 = precision_score(newytest11, predictions11, average='micro')
recall1 = recall_score(newytest11,predictions11,average='micro')
f11 = f1_score(newytest11, predictions11,average='micro')
print("PRECISION :", precision1)
print("RECALL :", recall1)
print("F1 SCORE:" ,f11)


PRECISION : 0.717373572593801
RECALL : 0.43394597261625756
F1 SCORE: 0.5407731919145339


In [88]:
import joblib
joblib.dump(classifier1,'model.pkl')
joblib.dump(features11,'features.pkl')
joblib.dump(vectorizer11,'tfidf_text.pkl')

['tfidf_text.pkl']